<a href="https://colab.research.google.com/github/Gattungswesen/MarkovChainPoems/blob/main/Markov_chains_poems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
The learning phase:
First, we generate the stochastic matrix using words from our sample text. This
is done by iterating through the text one word at a time. For each word, we
create a string of the preceding words, the length of which depends on the
number of words being considered for the state of the system. We refer to this
string as the Key. Using the Key, we look up a second table, which contains the
number of times that the key was followed by all other words, and we
increment the relevant value by 1.
To summarize, after this process we end up with a table of tables of
counts. The Key is used in a lookup in the first table, and the word following
the Key in the original poetry is used to lookup a count in the second table. The values in the 2nd table contain the actual frequencies with which that word
follows the Key.
The reason we organized our data this way is that it doesn’t require
saving the zero values in the matrix. Since most permutations of words will
never appear in the matrix, storing the matrix normally (as a 2-D array) would
require huge amounts of memory. By doing it like this, we can infer that any
items that didn’t have an entry in the tables must not have occurred in the
training data, and therefore have a zero percent chance of happening.```

Scraping using Beautifulsoup: 

In [ ]:
from bs4 import BeautifulSoup as bs
import re
import requests
import json
import threading
import queue
from sys import argv
import pickle
from collections import defaultdict
from math import ceil
# from os import exit
def getLink():
  while True:
    time = page = None
    try:
      time, page = remPgs.get(block=False)
    except queue.Empty:
     break
    print("getting pg tbl for pg %i from period %s"%(page, time))
    payload = {'page': str(page), 'preview': '1', 'school-period': time}
    r = requests.get(baseURL, params=payload)
    if (r.status_code != 200):
      print(f"page {page} failed. status code:{r.status_code}")
      continue
    for entry in r.json()['Entries']:
        link = entry['link']
        links.put((time, link))
def getPoems():
  while True:
    period = url = None
    try:
      period, url = links.get(block=False)
    except Empty:
        break
    r = requests.get(url)
    print("loaded page for %s"%(url))
    if (r.status_code != 200):
     print(f"poem lookup failed")
     continue
     soup = bs(r.text, 'html.parser')
     poem = '\r'.join([div.text.strip() for div in
     soup.find(class_=re.compile('o-poem')).find_all(style=re.compile('text-indent: -1em;'))])
     poem = poem.replace('\n', '')
     poem = poem.replace('\r', '\n')
     poems[period].append(poem)
     print("added poem in period: %s"%period)
# timePeriods = [('pre-1550', 52), ('1550-1780', 963), ('1781-1900', 1190),('1901-1950', 12984), ('1951-present', 27089)]
timePeriods = [('1951-present', 27089)]
baseURL = "https://www.poetryfoundation.org/ajax/poems"
numThreads = 50
remPgs = queue.Queue()
poemLock = threading.Lock()
poems = defaultdict(list)
links = queue.Queue()
for period, count in timePeriods:
  for page in range(ceil(count/20)):
    remPgs.put((period, page))
    threads = []
for _ in range(numThreads):
  thread = threading.Thread(target=getLink)
  threads.append(thread)
  thread.start()
for thread in threads:
  thread.join()
# while not links.empty():
# period, url = links.get()
# print(period, url)
# exit()
threads = []
for _ in range(numThreads):
  thread = threading.Thread(target=getPoems)
  threads.append(thread)
  thread.start()
for thread in threads:
  thread.join()
  f = open(argv[1], 'wb')
  pickle.dump(dict(poems), f)
  f.close()
# open('./strDmp.txt', 'w').write(str(pages))



```
The generation phase:
Once the stochastic matrix has been filled in, generating poetry is trivial. First,
we randomly select a Key to use as the beginning of the poem. Since this is
generated from the matrix, we know that this series of words must have
occurred at least once in the training text.
Once we have the first word(s), we repeatedly take the last words (again,
the number depends on how many words are being included in the state of the
system) [2]. This results in a table containing probabilities for the next word.
After randomly choosing the next word (based on the relative probabilities in
the matrix), the state of the system (corresponding to the last few words) is
updated, and the process repeats until the desired poem length is reached. The
length of the poem could be programmatically generated by finding the mean
and standard deviation of the length of a poem in the sample text, then using
that probability distribution to choose the length of each poem. We tried this but found that it just made the process of generating poems less reliable, since sometimes we ended up with very long or very short poems when we really wanted a relatively consistent poem length. As a result, we just manually
selected the length of the poems.```





Generating poems using Markov chains:

In [ ]:
import fileinput
from collections import Counter, defaultdict
import pprint
from random import choice, choices
from string import ascii_lowercase as ascii_lower
import re
# The stochastic matrix
# I'm using a dictionary data structure to avoid having to allocate a massive matrix that would mostly be full of 0s
matrix = defaultdict(Counter)
# This is the number of previous words to look at when choosing the next word
keyLen = 2
# process some training text (for this project, example poetry)
def learn(rawTxt):
 # split up the training text into individual words (and clean up the data a little by removing unwanted character)
 words = [word.strip().lower() for word in rawTxt.replace('\n', '').split(' ')]
 count = Counter()
 n = len(words)
 # for each unique group of 'keyLen' adjacent words in the 'words' array
 # count the relative frequencies of all possible following words
 for i in range(len(words)-keyLen-1):
   key = ' '.join([words[i + j].lower() for j in range(keyLen)])
   if (key == '' or key == ' '): continue
 matrix[key][words[i + keyLen]] += 1
# use the stochastic matrix to pick the next word given
def pickWord(currentKey = None):
  if currentKey == None or currentKey == '': return
  choice(list(matrix.keys()))
  words, counts = zip(*matrix[currentKey].items())
  return choices(words, weights=counts)[0] # this where the actual selection happens
# generate an 'nWords' text using the stochastic matrix's probabilities
def genText(nWords = 15):
 sentence = choice(list(matrix.keys())) + ' '
 for _ in range(nWords):
   key = ' '.join(sentence.split()[-keyLen:])
   sentence += pickWord(key) + ' '
   return sentence
def main():
 global keyLen
 trainTxt = ''.join([s for s in fileinput.input()])
 learn(trainTxt) # first we use the training data to generate the matrix
 for _ in range(3):
   resultText = genText(30).strip() # then we generate the text
   print(resultText + '\n\n')
 # print(resultText)
 if __name__=="__main__":
   main()